# Demonstration of Luma Module 
This notebook contain the implementation of the source code for each module in the EPISTEM land cover mapping framework

## Library import and earth engine initialization
If you have earth engine account you could used that to authenticate and initialize the earth engine. However, if you did not have the account, service account initialization is avaliable

In [ ]:
#This code is used if the notebook is implemented in github codespace. Just remove the (#)
!python -m pip install .. --quiet

In [ ]:
import ee 
import luma_ge

#Option 1: Manual authenticate using personal account
#Instructions for manual authentication
luma_ge.print_auth_instructions()
#uncomment the below line and follow earth engine authentication process
#luma_ge.authenticate_manually()

#Option 2: Autheticate using service account (json file)
service_account_path = '../auth/ee-epstm2024.json'
success = luma_ge.initialize_with_service_account(service_account_path)

if success:
    print("Earth Engine initialized with service account successfully!")
else:
    print("Service account initialization failed. Try to authenticate earth engine manually")

#Check authentication status
status = luma_ge.get_auth_status()
print(f"Initialized: {status['initialized']}")
print(f"Authenticated: {status['authenticated']}")
if status['project']:
    print(f"Project: {status['project']}")

In [ ]:
import geemap
from luma_ge.data_acquisition import Reflectance_Data, Reflectance_Stats
from luma_ge.helpers import get_aoi_from_gaul

## Module 1: Acquisition of Near-Cloud-Free Satellite Imagery

### System Response 1.1: Area of Interest Definition

In [ ]:
#Set the country and province for the AOI using GAUL admin boundaries
aoi = get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan")
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine

### System Response 1.2: Search and Filter Imagery
The EPISTEM source code supports Landsat mission data, ranging from Landsat 1 to Landsat 9. For Landsat 1 - 3, the avaliable data is corrected radiance reflectance. The Landsat 5-9 used here is collection 2 surface reflectance (SR) analysis ready data.

The retrival logic used here is as follow:
1. Retrive multispectral bands (band 1 - 7) from landsat collection 2 SR data (if avaliable)
2. Retrive thermal band from landsat collection 2 TOA data 
3. Create temporal composite for each data 
4. Stacked the final two data into a earth engine image (ee.image)

In [ ]:
#========== FIRST RETRIVE THE MULTISPECTRAL BAND===========
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2017-12-31'
#get the image collection and corresponding statistics
landsat_data, meta = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#create mosaic between image collection, and clip based on AOI
mosaic_landsat = landsat_data.mosaic().clip(aoi)
#Alternatively you can use temporal aggregation (ee reducer) to create mode cloudless imagery
median_landsat = landsat_data.median().clip(aoi)
#visualization parameter
l8_sr_visparam = {'min': 0,'max': 0.4,'gamma': [0.95, 1.1, 1],'bands':['NIR', 'RED', 'GREEN']}
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Median')
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)

In [ ]:
#retive thermal bands from TOA
thermal_bands, thermal_stats = optical_reflectance.get_thermal_bands(aoi, start, end, cloud_cover=40, compute_detailed_stats=False)
median_thermal = thermal_bands.median().clip(aoi)
thermal_vis = {
    'min': 286,
    'max': 300,
    'gammma': 0.4
}
#stacked all landsat bands
stacked_landsat = median_landsat.addBands(median_thermal)
#visualize the thermal bands and multispectral bands
Map.addLayer(median_thermal, thermal_vis, "Thermal Bands")
Map

### Image retrival report (optional)

In [ ]:
#intialize the statistic class
stats = Reflectance_Stats()
#get the retrival report and automatically print them
retrival_report = stats.get_collection_statistics(landsat_data, print_report=True)

### System Response 1.3: Imagery Download

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=stacked_landsat,
    description='Landsat_Median_composite_2017_Sumsel',
    folder='Earth Engine',
    fileNamePrefix='Landsat_Median_composite_2017_Sumsel',
    scale=30,
    region=aoi,  # or aoi.geometry()
    maxPixels=1e13
)
export_task.start()
import time

while export_task.active():
    print('Exporting... (status: {})'.format(export_task.status()['state']))
    time.sleep(10)

print('Export complete (status: {})'.format(export_task.status()['state']))

## Module 2:  Land-cover classification Scheme
Three approach are provided to handle classification scheme:
1. Upload a csv file 
2. Manual input the classification scheme
3. Use default classification scheme (RESTORE+ project)

### Import the module

In [ ]:
from luma_ge.classification_scheme import LULC_Scheme_Manager
#Initialize the LULC Scheme Manager
manager = LULC_Scheme_Manager()
print("Land Cover Classification Scheme Manager initialized!")
print(f"Current class count: {manager.get_class_count()}")
#Temporary function to display the classiifcation scheme in notebook
#Display current classification scheme
def display_classification_scheme(manager):
    """Display the current classification scheme in a readable format"""
    if not manager.has_classes():
        print("No classes defined yet.")
        return
    
    print("\n=== Current Classification Scheme ===")
    df = manager.get_dataframe()
    print(df.to_string(index=False))
    
    return df

# Display the scheme
df = display_classification_scheme(manager)

### System Response 2.1a: Upload Classification Scheme

In [ ]:
import pandas as pd
#Reset manager for CSV upload example
manager = LULC_Scheme_Manager()
#path to csv 
csv_path = "../data/Example_Classification_scheme.csv"

print("=== CSV Upload Process ===")

# Load the CSV
df = pd.read_csv(csv_path, sep=None, engine="python")
print("Loaded CSV:")
print(df)

# Auto-detect columns
id_col, name_col, color_col = manager.auto_detect_csv_columns(df)
print(f"\nAuto-detected columns:")
print(f"ID column: {id_col}")
print(f"Name column: {name_col}")
print(f"Color column: {color_col}")

In [ ]:
# Process CSV upload
success, message = manager.process_csv_upload(df, id_col, name_col, color_col)
if success:
    print(f"✅ {message}")
    
    # Finalize the upload
    success, message = manager.finalize_csv_upload()
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")
else:
    print(f"❌ {message}")

# Display the loaded scheme
display_classification_scheme(manager)

### System Response 2.1b: Manual Scheme Definition

In [ ]:
#Reset manager for manual input example
manager = LULC_Scheme_Manager()
#Manually add the class
print("=== Manual Class Addition ===")

#Example of class to add
classes_to_add = [
    (1, "Hutan Lahan Kering", "#0E6D0E"),
    (2, "Pertanian Lahan Kering", "#E8F800"),
    (3, "Permukiman", "#F81D00"),
    (4, "Badan Air", "#1512F3"),
    (5, "Pertanian Lahan Basah", "#")
]

for class_id, class_name, color_code in classes_to_add:
    success, message = manager.add_class(class_id, class_name, color_code)
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")

print(f"\nTotal classes: {manager.get_class_count()}")

In [ ]:
# Example: Edit an existing class
print("=== Editing a Class ===")

# Edit the first class (index 0)
class_to_edit = manager.edit_class(0)
if class_to_edit:
    print(f"Editing class: {class_to_edit}")
    
    # Update the class with new information
    success, message = manager.add_class(1, "HUtan Lahan Rendah", "#004D00")
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")

# Display updated scheme
display_classification_scheme(manager)

### System Response 2.1c: Template Classification Scheme

In [ ]:
# Reset manager for default scheme example
manager = LULC_Scheme_Manager()

print("=== Available Default Schemes ===")
default_schemes = manager.get_default_schemes()

for scheme_name, classes in default_schemes.items():
    print(f"\n{scheme_name}: {len(classes)} classes")
    for class_data in classes:
        print(f"  - ID {class_data['ID']}: {class_data['Class Name']} ({class_data['Color Code']})")

In [ ]:
# Load the RESTORE+ default scheme
scheme_name = "RESTORE+ Project"
success, message = manager.load_default_scheme(scheme_name)

if success:
    print(f"✅ {message}")
else:
    print(f"❌ {message}")

# Display the loaded scheme
display_classification_scheme(manager)

### System Response 2.2: Download classification scheme

In [ ]:
print("=== Export Classification Scheme ===")
#Convert the selected  classification scheme manager to dataframe
classification_df = manager.get_dataframe()
print("Classification DataFrame:")
print(classification_df)
#Save the file
output_path = '../Selected_LC_Classification_Scheme.csv'
classification_df.to_csv(output_path, index=False)
print(f"\n✅ Classification scheme saved to: {output_path}")

# Module 3: Generate Region Of Interest
Three methods to generate ROI are supported in EPISTEM platform:
1. **Upload Training Data** - Upload your own shapefile
2. **On-screen Sampling** - Create samples using interactive map
3. **Default Reference Data** - Use Epistem's default training data

## Library Import and Setup

## System Response 3.1 Prerequisite Check

In [ ]:
print("=== Checking Prerequisites ===")
#Load from previous module
#From Module 1 - AOI data
try:
    AOI = aoi
    print("✅ AOI from Module 1 is available")
    aoi_available = True
except:
    print("❌ AOI data not available, please run Module 1 first")
    aoi_available = False

#From Module 2 - Classification scheme
try:
    
    # For demonstration, create sample classification scheme
    LULCTable = classification_df
    print("✅ Classification scheme from Module 2 is available")
    print(f"   - Number of classes: {len(LULCTable)}")
    scheme_available = True
except:
    print("❌ Classification scheme not available, please run Module 2 first")
    scheme_available = False

if aoi_available and scheme_available:
    print("\n✅ All prerequisites met! You can proceed with training data collection.")
else:
    print("\n❌ Prerequisites not met. Please complete previous modules first.")

## System Response 3.2 ROI Upload and content Verification

In [ ]:
# Modul 3a 
# Import modules and functions
import ee
import pandas as pd
from luma_ge.sample_data import SyncTrainData, SplitTrainData

In [ ]:
# ----- Data Input -----
# 1. Decision to upload data
UploadTrainData = True # set as 'true' to upload your own training data shapefile
# set as 'false' to either add train data by sampling on screen or use default training data

# 2. Training data file path (if UploadTrainData is true)
TrainVectPath  = '../data/Training_Sumsel_Data.shp'
TrainField = 'ID' 
        # Load and process training data
TrainDataDict = SyncTrainData.LoadTrainData(
            landcover_df=LULCTable,
            aoi_geometry=AOI,
            training_shp_path=TrainVectPath
        )

In [ ]:
# ----- System response 3.2.a -----
# Set class field
TrainDataDict = SyncTrainData.SetClassField(TrainDataDict, TrainField)

# Validate classes
TrainDataDict = SyncTrainData.ValidClass(TrainDataDict, 1)

    # Check sample sufficiency
TrainDataDict = SyncTrainData.CheckSufficiency(TrainDataDict, min_samples=20)

    # Filter by AOI
TrainDataDict = SyncTrainData.FilterTrainAoi(TrainDataDict)

    # Create training data table
table_df, total_samples, insufficient_df = SyncTrainData.TrainDataRaw(
    training_data=TrainDataDict.get('training_data'),
    landcover_df=TrainDataDict.get('landcover_df'),
    class_field=TrainDataDict.get('class_field'))

#Summary result
vr = TrainDataDict.get('validation_results', {})

print("=" * 70)
print("TRAINING DATA SUMMARY")
print("=" * 70)
print(f"Total training points loaded     : {vr.get('total_points', 'N/A')}")
print(f"Points after class filtering     : {vr.get('points_after_class_filter', 'N/A')}")
print(f"Valid points (inside AOI)        : {vr.get('valid_points', 'N/A')}")
print(f"Invalid classes found            : {len(vr.get('invalid_classes', []))}")
print(f"Points outside AOI               : {len(vr.get('outside_aoi', []))}")
print("=" * 70)

    # --- Display the main table ---
if table_df is not None and not table_df.empty:
        display_df = table_df.copy()
        if 'Percentage' in display_df.columns:
            display_df['Percentage'] = display_df['Percentage'].apply(
                lambda x: f"{x:.2f}%" if isinstance(x, (int, float)) else x
            )
        display(display_df)
else:
        print("No valid training data available to display.")

## System Response 3.2 Default ROI

In [ ]:
print(" Loading default reference training data...")
TrainEePath = 'projects/ee-rg2icraf/assets/Indonesia_lulc_Sample'
TrainField = 'kelas'

# Stopgap solution: Rename 'Land Cover Class' column to 'LULC_Type' if it exists
if 'Land Cover Class' in LULCTable.columns:
    LULCTable = LULCTable.rename(columns={'Land Cover Class': 'LULC_Type'})
    print("Column 'Land Cover Class' renamed to 'LULC_Type'")

    
try:
    print("Loading reference training data from Earth Engine...")
        
        # Load training data
    TrainDataDict = SyncTrainData.LoadTrainData(
            landcover_df=LULCTable,
            aoi_geometry=AOI,
            training_ee_path=TrainEePath
        )
        
    print("Processing and validating reference data...")
        
        # Set class field
    TrainDataDict = SyncTrainData.SetClassField(TrainDataDict, TrainField)
        
        # Validate classes
    TrainDataDict = SyncTrainData.ValidClass(TrainDataDict)
        
        # Check sufficiency
    TrainDataDict = SyncTrainData.CheckSufficiency(TrainDataDict, min_samples=20)
        
        # Filter by AOI
    TrainDataDict = SyncTrainData.FilterTrainAoi(TrainDataDict)
        
        # Create summary table
    table_df, total_samples, insufficient_df = SyncTrainData.TrainDataRaw(
            training_data=TrainDataDict.get('training_data'),
            landcover_df=TrainDataDict.get('landcover_df'),
            class_field=TrainDataDict.get('class_field')
        )
        
    print("✅ Reference training data loaded and processed successfully!")
    print(f"Total samples: {total_samples}")
        
        # Display summary table
    display(table_df)
        
        # Store final training data
    TrainDataFinal = TrainDataDict.get('training_data')
        
        # Show validation results
    vr = TrainDataDict.get('validation_results', {})
    print(f"\nValidation Results:")
    print(f"- Total points loaded: {vr.get('total_points', 'N/A')}")
    print(f"- Points after class filter: {vr.get('points_after_class_filter', 'N/A')}")
    print(f"- Valid points (within AOI): {vr.get('valid_points', 'N/A')}")
    print(f"- Invalid classes: {len(vr.get('invalid_classes', []))}")
        
except Exception as e:
        print(f"❌ Error loading reference data: {e}")
        TrainDataFinal = None

# Module 4: Region of Interest Separability Analysis

## Library Import and Setup

In [ ]:
#Import the sample quality functions
from luma_ge.sample_data_quality import sample_quality, spectral_plotter
#if there's error in the import, uncomment the below line to install the PyCRS package
#!pip install PyCRS

## System Response 4.1 Computing Separability Analysis

In [ ]:
roi_path = '../data/Training_Sumsel_Data.shp'
labeled_roi = geemap.shp_to_ee('../data/Training_Sumsel_Data.shp')
# labeled_roi = geemap.gdf_to_ee(TrainDataFinal)

#Conduct the analysis
analyzer = sample_quality(training_data=labeled_roi, 
    image= stacked_landsat, 
    class_property='ID',           # Column with numeric IDs (1, 2, 3, etc.)
    region= aoi,
    class_name_property='LC_Name'          # Column with names ('Forest', 'Urban', 'Water', etc.)
)
# Extract spectral values
pixel_extract = analyzer.extract_spectral_values(scale=100, max_pixels_per_class=5000)
samples_statistic = analyzer.sample_stats()
sample_df = analyzer.get_sample_stats_df()
display(sample_df)
#Sample statistic

In [ ]:
#Sample statistic (pixel value extracted from the imagery)
pixel_stats = analyzer.sample_pixel_stats(pixel_extract)
pixel_stats_df = analyzer.get_sample_pixel_stats_df(pixel_extract)
display(pixel_stats_df)

In [ ]:
#Perform separability Analysis (iether using Transformed Divergence or Jeffries Matutista )
separability_analysis = analyzer.get_separability_df(pixel_extract, method='TD')
display(separability_analysis)

In [ ]:
#Get the lowest separability
lowest_sep = analyzer.lowest_separability(pixel_extract)
display(lowest_sep)

In [ ]:
# Overall separability summary
sep_summary = analyzer.sum_separability(pixel_extract)
print("Overall Separability Statistics:")
display(sep_summary)

## System Response 4.2 Sample Visualization

In [ ]:
#Box plot to detect outlier
ploter = spectral_plotter(analyzer)
box = ploter.plot_boxplot(pixel_extract)
for fig in box:
    fig.show()

In [ ]:
#static scatter plot
stat_plot = ploter.static_scatter_plot(pixel_extract, x_band='NIR', y_band='RED', add_ellipse=True)

In [ ]:
#3D scatter plot
multi_d_scater = ploter.scatter_plot_3d(pixel_extract)
multi_d_scater

# Module 6: Land Cover Classification

## Library Import

In [ ]:
from luma_ge.classification import FeatureExtraction, Generate_LULC
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## System Response 6.2 Classification

In [ ]:
#Perform Training Test Split
features = FeatureExtraction()
strafied_train, stratified_test = features.stratified_split(labeled_roi, stacked_landsat, 
                            class_prop='ID', train_ratio=0.5)
classifier = Generate_LULC()
print("Performing Classification...")
#Multiclass hard classification
classification_map, trained_model = classifier.hard_classification(strafied_train, class_property='ID', image=stacked_landsat,
                                                          ntrees=300, min_leaf=2, return_model=True)
# Evaluate model performance
print("Evaluating model performance...")

try:
    accuracy_metrics = classifier.evaluate_model(
        trained_model=trained_model,
        test_data=stratified_test,
        class_property='ID'
    )
    
    print("✓ Model evaluation completed")
    
except Exception as e:
    print(f"❌ Error in model evaluation: {e}")                                                    


## System Response 6.3 Model Evaluation

In [ ]:
# Display accuracy results
print("=== Model Performance Summary ===")
print(f"Overall Accuracy: {accuracy_metrics['overall_accuracy']:.4f} ({accuracy_metrics['overall_accuracy']*100:.2f}%)")
print(f"Kappa Coefficient: {accuracy_metrics['kappa']:.4f}")
print(f"Overall G-Mean: {accuracy_metrics['overall_gmean']:.4f}")

print("\n=== Per-Class Metrics ===")
#Class Dataframe
metrics_df = pd.DataFrame({
    'Precision': accuracy_metrics['precision'],
    'Recall': accuracy_metrics['recall'],
    'F1-Score': accuracy_metrics['f1_scores'],
    'G-Mean': accuracy_metrics['gmean_per_class']
})

# Round to 4 decimal places
metrics_df = metrics_df.round(4)

display(metrics_df)

In [ ]:
# Visualize confusion matrix
confusion_matrix = np.array(accuracy_metrics['confusion_matrix'])
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, 
            annot=True, 
            fmt='d', 
            cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.tight_layout()
plt.show()

In [ ]:
# Get feature importance
print("Analyzing feature importance...")

try:
    importance_df = classifier.get_feature_importance(trained_model)
    print("✓ Feature importance analysis completed")
    
    display(importance_df)
    
except Exception as e:
    print(f"❌ Error in feature importance analysis: {e}")
# Visualize feature importance
plt.figure(figsize=(10, 6))

# Create bar plot
bars = plt.bar(importance_df['Band'], importance_df['Importance'])

# Add value labels on bars
for bar, value in zip(bars, importance_df['Importance']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{value:.1f}%', ha='center', va='bottom')

plt.title('Feature Importance by Spectral Band')
plt.xlabel('Landsat 8 Bands')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Visualization

In [ ]:
# === Load Classification Scheme ===
scheme = pd.read_csv("../Selected_LC_Classification_Scheme.csv", sep=None, engine="python")
classes = [str(x).strip() for x in scheme["Land Cover Class"].tolist()]
palette = [str(x).strip() for x in scheme["Color Palette"].tolist()]
ids = scheme["ID"].tolist()
legend_dict = dict(zip(classes, palette))

# === Visualization Parameters ===
vis_params = {
    "min": min(ids),
    "max": max(ids),
    "palette": palette
}

# === Create geemap Map ===
Map = geemap.Map() 
Map.centerObject(aoi, 7)
Map.addLayer(classification_map, vis_params, "LULC Classification")

# # === Add Legend ===
Map.add_legend(
    title="Land Cover Classification", 
    legend_dict=legend_dict
    )

# Display
Map


# Module 7: Thematic Accuracy Assessment

## System Response 7.3 Thematic Accuracy Assessment

In [ ]:
from luma_ge.accuracy import Thematic_Accuracy_Assessment
#Initialize the accuracy assessment class
accuracy_assessor = Thematic_Accuracy_Assessment()
print("✓ Thematic Accuracy Assessment class initialized")
print(f"Supported metrics: {accuracy_assessor.supported_metrics}")

In [ ]:
validation_data = geemap.shp_to_ee("../data/Evaluation_Sumsel_data.shp") 

# === 2. Create Assessment Object ===
assessor = Thematic_Accuracy_Assessment()

# === 3. Run Accuracy Assessment ===
success, results = assessor.run_accuracy_assessment(
    lcmap=classification_map,
    validation_data=validation_data,
    class_property='LULC_ID',   #Validation ID column
    scale=30
)

# === 4. Display Results ===
if success:
    print("=== Thematic Accuracy Results ===")
    summary = assessor.format_accuracy_summary(results)
    print("Overall Accuracy :", summary['overall_accuracy'])
    print("Kappa Coefficient:", summary['kappa'])
    print("95% CI          :", summary['confidence_interval'])
    print("Samples Used     :", summary['sample_size'])
else:
    print("Error:", results["error"])
